In [1]:
! pip install frictionless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
  Created wheel for petl: filename=petl-1.7.12-py3-none-any.whl size=226948 sha256=81815a2e7679e7dd3f6f372c6e7dea29563f322b9d873dfa981f40650273a79c
  Stored in directory: /root/.cache/pip/wheels/17/a2/7a/68707dd0e3b3ce2a2acfcec8e16eb8464018a4a57eca1fd8fb
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3570 sh

In [2]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time
import os
import frictionless
from getpass import getpass
from ftplib import FTP_TLS
from ftplib import FTP
import ftplib
import ast

In [9]:
import pandas as pd
import re
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

def extract_id(fileName):
    ogd_id = fileName.partition('_')[0]
    return ogd_id.replace('ogd', '')

def getInformation(url):

  csvList = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))

  for elem in x:
    stringElem = str(elem)

    if 'csv' and 'ogd' in stringElem:
      fileName = re.search('href="(.*)"', stringElem).group(1)
      csvList.append([fileName, extract_id(fileName)])

  df = pd.DataFrame(csvList, columns= ['fileName', 'ogd_id'])
  print(df)
  return df
  #df.to_csv('files/info.csv')

df = getInformation(url)

                                            fileName  ogd_id
0                                          /BFE/ogd/  /BFE//
1  ogd35_schweizerische_elektrizitaetsbilanz_mona...      35
2                                     ogd39_epec.csv      39
3                     ogd47_vollzugsresultate_pw.csv      47
4                ogd47_vollzugsresultate_pw_2020.csv      47
5                    ogd61_vollzugsresultate_lnf.csv      61
6               ogd61_vollzugsresultate_lnf_2020.csv      61
7           ogd90_potenzialstudien_waermenutzung.csv      90


-----------------------------------------------------------------------------------------------

Test the original datapackages:

In [4]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
url = 'https://www.uvek-gis.admin.ch/BFE/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

  # load original data package
  folderPath = url_ogd + row['ogd_id'] + '/'
  datapackagePath = os.path.join(folderPath, dataPackage)

  # new file path --> to staging folder
  stagingFilePath = url_ogd + 'staging/' + row['fileName']

  # store the response of URL
  response = urlopen(datapackagePath)

  # storing the JSON response
  data_json = json.loads(response.read())

  print('original datapackage: ')
  print(data_json)
  print('===========================')

  # path for temporary datapackage
  tmpJSON_path = 'tmpPackage.json'

  # save the file as data.json
  with open(tmpJSON_path, 'w') as f:
    json.dump(data_json, f)

  report = frictionless.validate(tmpJSON_path)

  if report.valid:
    print("Valid.\n")
  else:
    print("Not valid!\n")




original datapackage: 
{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/35/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Lau

Test the adjusted files:

In [6]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
url = 'https://www.uvek-gis.admin.ch/BFE/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

  # load original data package
  folderPath = url_ogd + row['ogd_id'] + '/'
  datapackagePath = os.path.join(folderPath, dataPackage)

  # new file path --> to staging folder
  stagingFilePath = url_ogd + 'staging/' + row['fileName']

  # store the response of URL
  response = urlopen(datapackagePath)

  # storing the JSON response
  data_json = json.loads(response.read())

  print('original datapackage: ')
  print(data_json)


  # change source file
  jsonAsString = str(data_json)
  jsonAsString = jsonAsString.replace(os.path.join(folderPath, row['fileName']), stagingFilePath)
  updatedJSON = ast.literal_eval(jsonAsString)

  print('---------------------------')
  print('adjusted datapackage: ')
  print(updatedJSON)
  print('===========================')

  # path for temporary datapackage
  tmpJSON_path = 'tmpPackage.json'

  # save the file as data.json
  with open(tmpJSON_path, 'w') as f:
    json.dump(updatedJSON, f)

  report = frictionless.validate(tmpJSON_path)

  if report.valid:
    print("Valid.\n")
  else:
    print("Not valid!\n")




original datapackage: 
{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/35/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Lau